# Model development

In [1]:
from src.models.model import *

# Create an NER model from a base pretrained model
model_loader = ModelLoader('UFNLP/gatortron-base')
NER_model, tokenizer, config = model_loader.load_model(num_labels = 10)

Some weights of the model checkpoint at UFNLP/gatortron-base were not used when initializing MegatronBertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing MegatronBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MegatronBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MegatronBertForTokenClassification were not initialized from 